# Scrapping Ulasan Aplikasi TikTok

In [1]:
!pip install google-play-scraper pandas
import pandas as pd

In [2]:
from google_play_scraper import reviews_all, Sort
from google_play_scraper import app
import numpy as np
import pandas as pd

In [ ]:
# Inisialisasi DataFrame kosong
df_busu = pd.DataFrame(columns=['review'])

# Loop untuk mengambil data
for _ in range(100):
    result = reviews_all(
        'com.ss.android.ugc.trill',
        sleep_milliseconds=0,  # default 0
        lang='id',  # default 'en'
        country='id',  # default 'us'
        sort=Sort.NEWEST  # default Sort.MOST_RELEVANT
    )

    # Konversi hasil ke DataFrame sementara
    df_temp = pd.DataFrame(np.array(result), columns=['review'])
    df_temp = df_temp.join(pd.DataFrame(df_temp.pop('review').tolist()))

    # Gabungkan dengan data sebelumnya
    df_busu = pd.concat([df_busu, df_temp], ignore_index=True)

    print(f"Total data saat ini: {len(df_busu.index)}")

    # Berhenti jika jumlah data tepat 10.000
    if len(df_busu.index) >= 10000:
        df_busu = df_busu.iloc[:10000]  # Pastikan hanya 10.000 data yang tersimpan
        break

# Simpan hasil ke CSV
df_busu.to_csv('tiktok_reviews_10000.csv', index=False, encoding='utf-8')

print("Data berhasil disimpan dengan 10.000 sampel.")


Total data saat ini: 360000
Data berhasil disimpan dengan 10.000 sampel.


In [ ]:
df_busu[['userName', 'score', 'at', 'content']].head()

,userName,score,at,content
0,Pengguna Google,5.0,2025-02-13 15:19:07,Sangat bagus
1,Pengguna Google,3.0,2025-02-13 15:18:59,Kenapa tidak bisa masuk trs tolong diperbaiki
2,Pengguna Google,5.0,2025-02-13 15:18:33,ko ga bisa di buka
3,Pengguna Google,1.0,2025-02-13 15:17:29,stiker musiknya mana woi dev poke
4,Pengguna Google,5.0,2025-02-13 15:17:27,Tik tok kan Pasti nya Asyik Dong


### Loading Dataset

In [1]:
import pandas as pd
ulasan_tiktok = pd.read_csv("tiktok_review_sample.csv")

# Now you can call head() on the DataFrame
ulasan_tiktok.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,07be1285-e2cb-4228-8721-21c91af3e925,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,mantap,5,0,36.3.3,2024-09-03 07:51:56,NaN,NaN,36.3.3
1,ae0206fa-fc6b-44ad-9c58-2b495f6dc32e,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Banyak event Afiliasi yang mengharuskan pemena...,4,1,35.6.3,2024-08-03 10:06:16,NaN,NaN,35.6.3
2,cca03e4e-ee96-41b5-98a7-3906af44b0dc,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Aplikasi yg sangat menguntungkan, dan banyak p...",5,0,NaN,2025-01-28 12:07:33,NaN,NaN,NaN
3,0eab3844-8043-4948-9154-32d925cfeb7d,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,bug mulu gabisa dibuka harus download ulang trus,1,0,NaN,2024-12-14 02:47:12,NaN,NaN,NaN
4,5455a81a-e5f0-4a93-99a5-07872688c3b1,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Lumayan keren lh,1,0,NaN,2024-11-12 10:11:01,NaN,NaN,NaN


In [2]:
ulasan_tiktok.shape

(10000, 11)

In [3]:
ulasan_tiktok.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   reviewId              10000 non-null  object
 1   userName              10000 non-null  object
 2   userImage             10000 non-null  object
 3   content               10000 non-null  object
 4   score                 10000 non-null  int64 
 5   thumbsUpCount         10000 non-null  int64 
 6   reviewCreatedVersion  6090 non-null   object
 7   at                    10000 non-null  object
 8   replyContent          50 non-null     object
 9   repliedAt             50 non-null     object
 10  appVersion            6090 non-null   object
dtypes: int64(2), object(9)
memory usage: 859.5+ KB


In [4]:
ulasan_tiktok.isnull().sum()

,0
reviewId,0
userName,0
userImage,0
content,0
score,0
thumbsUpCount,0
reviewCreatedVersion,3910
at,0
replyContent,9950
repliedAt,9950


In [6]:
ulasan_tiktok['replyContent'].head(5)

,replyContent
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN


In [7]:
print(ulasan_tiktok.duplicated().sum())

0


In [8]:
ulasan_tiktok['replyContent'].fillna('No Reply', inplace=True)
ulasan_tiktok['repliedAt'].fillna('No Date', inplace=True)


<ipython-input-8-84cdf489502d>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  ulasan_tiktok['replyContent'].fillna('No Reply', inplace=True)
<ipython-input-8-84cdf489502d>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True

In [10]:
most_common_version = ulasan_tiktok['appVersion'].mode()[0]
ulasan_tiktok['appVersion'].fillna(most_common_version, inplace=True)
ulasan_tiktok['reviewCreatedVersion'].fillna(most_common_version, inplace=True)


<ipython-input-10-60a4b25a043c>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  ulasan_tiktok['appVersion'].fillna(most_common_version, inplace=True)
<ipython-input-10-60a4b25a043c>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inp

In [11]:
ulasan_tiktok.isnull().sum()

,0
reviewId,0
userName,0
userImage,0
content,0
score,0
thumbsUpCount,0
reviewCreatedVersion,0
at,0
replyContent,0
repliedAt,0


In [12]:
ulasan_tiktok.to_csv('cleaned_ulasan_tiktok.csv', index=False)
